# StoneGate Macro Wizard Notebook Import Test

This notebook is primarily an *import fixture* for the Macro Wizard: the macro is embedded in notebook metadata at `metadata.stonegate.macros`.

If you want to see the full generated Python script, either:
- Export as Notebook/Python from the Macro Wizard, or
- Run the next cell (it contains a runnable Python version using the StoneGate SDK: `import stonegate_api`).

In [ ]:
# Runnable macro (requires StoneGate SDK: import stonegate_api)
import asyncio

import sys
from pathlib import Path

# Make StoneGate SDK importable (repo-local) without a pip install.
repo_root = Path.cwd().resolve()
sdk_path = None
for _ in range(8):
    candidate = repo_root / 'sdk' / 'python' / 'stonegate_sdk'
    if candidate.exists():
        sdk_path = candidate
        break
    candidate = repo_root / 'tools' / 'sdk_sources'
    if candidate.exists():
        sdk_path = candidate
        break
    repo_root = repo_root.parent

if sdk_path is None:
    raise FileNotFoundError('Could not locate StoneGate SDK. Run from inside the repo or install via: python3 -m pip install -e sdk/python/stonegate_sdk')

sys.path.insert(0, str(sdk_path))

import stonegate_api as sg

sg.WS_URL = 'ws://localhost:8080/status'

SAFE_TARGETS = {
    'laser_controller': {'intensity': 0},
    'ln2': {'flow_rate_Lmin': 0, 'temperature_K': 77},
}

async def run_macro():
    active_recording_ids = set()
    try:
        await sg.device_action('ln2', {'set': {'flow_rate_Lmin': 1.5, 'temperature_K': 80}})
        await sg.wait_for_stable(device_id='ln2', metric='temperature_K', tolerance=0.2, window_s=2.0, consecutive=3, timeout_s=30.0)
        await asyncio.sleep(0.5)

        rid = await sg.record_start({'tag': 'macro_wizard_notebook_test'})
        active_recording_ids.add(rid)
        try:
            await sg.device_action('laser0', {'set': {'intensity': 0.2, 'phase_rad': 0.5}})
            await asyncio.sleep(0.25)

            latest = await sg.get_latest_number('tc0', 'temperature_C')
            if sg.eval_condition(latest, '>', 30):
                await sg.device_action('ln2', {'set': {'flow_rate_Lmin': 2.5}})
            else:
                await sg.device_action('ln2', {'set': {'flow_rate_Lmin': 1.0}})

            for _ in range(5):
                latest = await sg.get_latest_number('tc0', 'temperature_C')
                if not sg.eval_condition(latest, '>', 25):
                    break
                await asyncio.sleep(0.5)
                await sg.device_action('ln2', {'set': {'temperature_K': 78}})
        finally:
            await sg.record_stop(rid)
            active_recording_ids.discard(rid)
    finally:
        await sg.apply_safe_state(active_recording_ids, SAFE_TARGETS)

# In Jupyter, top-level await works:
await run_macro()